In [1]:
using StatsBase
using Combinatorics
using Plots

using Pkg; Pkg.activate(".")
using HypergraphModularity
# parameters

n = 100
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 4

Ω = buildΩ(ω, α0, kmax)

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


(::HypergraphModularity.var"#Ω#8"{HypergraphModularity.var"#Ω#3#9"{typeof(ω),Int64}}) (generic function with 1 method)

In [2]:
# sample from the HSBM with these parameters, restricting to hyperedges of size no more than kmax
H = sampleSBM(Z, ϑ, Ω;α=α0, kmax=kmax, kmin = 1)

HypergraphModularity.hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [3, 1, 3, 5, 2, 9, 2, 4, 4, 6  …  4, 3, 3, 6, 4, 1, 5, 1, 4, 5]


In [3]:
# Create a random clustering with k clusters
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω;α=α0)

-1259.180686193258

In [4]:
# still works because we haven't added zero-padding. 

for i = 1:4, p in partitions(4, i)
    print(p, " ")
    println(Ω(p;α=α0,mode="partition"))
end

# an interesting note from this is that the harmonic mean doesn't actually respect the majorization order....

[4] 1.0668131044536959e-7
[3, 1] 9.92781458758405e-8
[2, 2] 1.0668131044536959e-7
[2, 1, 1] 8.970793152807994e-8
[1, 1, 1, 1] 7.54350780417881e-8


In [5]:
@time T1 = first_term_eval(H,c,Ω;α=α0)

  0.000204 seconds (1.78 k allocations: 92.375 KiB)


-1259.180686193258

In [6]:
# Here's a faster way to calculate it...for now requires a different way to store the hypergraph and Omega

@time Hyp, w = hyperedge_formatting(H)

@time T2 = first_term_v2(Hyp,w,c,Ω;α=α0)

T1 ≈ T2

  0.095890 seconds (167.42 k allocations: 8.631 MiB)
  0.024219 seconds (31.03 k allocations: 1.588 MiB)


true